In [ ]:
# Import packages
import xarray as xr
import numpy as np
import cf_xarray
import matplotlib.pyplot as plt 
import cartopy.crs as ccrs
import cartopy.feature as cf
import rasterio 
import rioxarray as rio
import glob 
from os.path import basename